In [1]:
# Bibliotecas para manipulacao das planilhas
import pandas as pd
import math
import numpy as np

In [2]:
def classificar_risco(n):
    if n <= 400:           # Baixo risco
        return 0
    elif 400 < n <= 1000:  # Médio risco
        return 1
    elif 1000 < n <= 2500: # Alto risco
        return 2
    else:                  # Risco crítico
        return 3

In [3]:
def estacao_do_ano(data):
    ano = data.year
    if (pd.Timestamp(f"{ano}-12-21") <= data) or (data < pd.Timestamp(f"{ano}-03-21")): # Verão
        return 0
    elif pd.Timestamp(f"{ano}-03-21") <= data < pd.Timestamp(f"{ano}-06-21"):           # Outono
        return 1
    elif pd.Timestamp(f"{ano}-06-21") <= data < pd.Timestamp(f"{ano}-09-23"):           # Inverno
        return 2
    else:                                                                               # Primavera
        return 3

In [4]:
def calcular_janela_precipitacao(df, janela=5):
    """
    Calcula a precipitação acumulada dos últimos 'janela' dias considerando o período de 16:00 UTC a 16:00 UTC.

    A precipitação acumulada será mostrada desde o primeiro dia disponível.

    Parâmetros:
    - df: DataFrame contendo a coluna 'precipitacao_total'.
    - janela: Número de dias para calcular a soma da precipitação (padrão: 5 dias).

    Retorna:
    - DataFrame com a coluna adicional 'precipitacao_acumulada'.
    """

    if 'precipitacao_total' not in df.columns:
        raise ValueError("Erro: A coluna 'precipitacao_total' não está no DataFrame. Verifique os dados de entrada.")

    # Ajustar a data para que o ciclo vá das 16:00 UTC de um dia até as 16:00 UTC do dia seguinte
    df['DATA_16UTC'] = (df.index - pd.Timedelta(hours=16)).date  # Define o novo dia com base em 16:00 UTC

    # Criar um DataFrame diário somando a precipitação no intervalo 16UTC-16UTC
    df_diario = df.groupby('DATA_16UTC')['precipitacao_total'].sum().to_frame()

    # Aplicar a soma acumulada dos últimos 'janela' dias desde o primeiro dia disponível
    df_diario['precipitacao_acumulada'] = df_diario['precipitacao_total'].rolling(window=janela, min_periods=1).sum()

    # Mapear os valores para todas as horas do mesmo ciclo 16UTC-16UTC
    df['precipitacao_acumulada'] = df['DATA_16UTC'].map(df_diario['precipitacao_acumulada'])

    # Remover a coluna auxiliar
    df.drop(columns=['DATA_16UTC'], inplace=True)

    return df

In [5]:
def calcula_nesterov(df):
    # Garantir que o índice seja datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index, errors='coerce')

    # Criar a coluna do mês a partir do índice
    df['mes'] = df.index.month

    # Definir fator de ajuste sazonal
    fatores_mensais = {1: 0.8, 2: 0.9, 3: 1.0, 4: 1.2, 5: 1.5, 6: 1.7,
                        7: 2.0, 8: 2.5, 9: 2.8, 10: 2.5, 11: 1.8, 12: 1.2}
    df['fator_mes'] = df['mes'].map(fatores_mensais)

    # Cálculo do déficit de umidade
    df['deficit_umidade'] = (df['temperatura_med'] - df['temperatura_orvalho']).clip(lower=0)

    # Ajuste da precipitação acumulada
    df['precip_soma'] = df['precipitacao_acumulada'].rolling(window=5, min_periods=1).sum()

    # Redução de 50% em vez de resetar para zero se a precipitação for maior que 10mm
    df.loc[df['precipitacao_acumulada'] > 10, 'precip_soma'] *= 0.5

    # Limitar a precipitação acumulada para evitar valores extremos
    df['precip_soma'] = df['precip_soma'].clip(upper=30)

    # Cálculo do Índice de Nesterov ajustado
    df['nesterov'] = df['deficit_umidade'] * (df['precip_soma'] + 1) * df['fator_mes'] * 5

    # Garantir que os valores não sejam negativos
    df['nesterov'] = df['nesterov'].clip(lower=0)

    return df.drop(df.columns[[9, 10]], axis=1)


In [6]:
def renameColumns(df_aux):

    df_aux.rename(columns={df_aux.columns[0]: 'DATA', df_aux.columns[1]: 'HORA', df_aux.columns[2]: 'precipitacao_total',
                           df_aux.columns[3]: 'pressao_atmosferica', df_aux.columns[6]: 'radiacao',
                           df_aux.columns[7]: 'temperatura_bulbo',  df_aux.columns[8]: 'temperatura_orvalho',
                           df_aux.columns[9]: 'temperatura_max', df_aux.columns[10]: 'temperatura_min',
                           df_aux.columns[15]: 'umidade'}, inplace=True)

    return df_aux

In [7]:
def preencher_media_historica(df, usar_mediana=False):
    """
    Preenche os valores ausentes (NaN e -9999) com a média histórica do mesmo dia e horário ao longo dos anos.
    Se 'usar_mediana' for True, a mediana será usada ao invés da média para evitar influência de outliers.
    """
    df['mes-dia'] = df.index.strftime('%m-%d')
    df['hora'] = df['HORA'].astype(str).str.zfill(5).str[:2]  # Normaliza a hora (ex: 1600 UTC -> 16)

    colunas_numericas = ['precipitacao_total', 'pressao_atmosferica', 'radiacao',
                         'temperatura_bulbo', 'temperatura_orvalho', 'temperatura_max',
                         'temperatura_min', 'umidade']

    # Substitui valores inválidos por NaN
    df[colunas_numericas] = df[colunas_numericas].replace(-9999, np.nan)

    # Escolhe a função de agregação (média ou mediana)
    func_agregacao = np.median if usar_mediana else np.mean

    # Primeira tentativa: Preencher com a média/mediana do mesmo dia e hora
    for coluna in colunas_numericas:
        df[coluna] = df.groupby(['mes-dia', 'hora'])[coluna].transform(lambda x: x.fillna(func_agregacao(x.dropna())))

    # Segunda tentativa: Se ainda houver valores NaN, preencher com a média/mediana geral do horário ao longo do ano
    for coluna in colunas_numericas:
        df[coluna] = df.groupby('hora')[coluna].transform(lambda x: x.fillna(func_agregacao(x.dropna())))

    # Terceira tentativa: Se ainda houver NaN, preencher com a média/mediana do mês + hora
    for coluna in colunas_numericas:
        df[coluna] = df.groupby([df.index.month, 'hora'])[coluna].transform(lambda x: x.fillna(func_agregacao(x.dropna())))

    # Remover colunas auxiliares
    df.drop(columns=['mes-dia', 'hora'], inplace=True)

    return df

In [8]:
def load_df(years, usar_mediana=False):

    dataframes = []

    # Carrega todas as planilhas em dataframes
    for year in years:
        filePath = f'INMET_S_PR_A819_CASTRO_01-01-{year}_A_31-12-{year}.CSV'

        try:
            df_aux = pd.read_csv(filePath, sep=';', decimal=',', encoding='latin1', skiprows=8)
            df_aux = renameColumns(df_aux)
            dataframes.append(df_aux)

        except Exception as e:
            print(f'ERRO: Falha ao carregar DataFrame para o ano {year}')

    # Concatena os dataframes
    df = pd.concat(dataframes, ignore_index=True, join='outer')

    # Exclui colunas indesejadas
    df = df.drop(df.columns[[4, 5, 11, 12, 13, 14, 16, 17, 18, 19]], axis=1)

    # Padroniza datas e horários
    df['DATA'] = df['DATA'].str.replace('/', '-')
    df['DATA'] = pd.to_datetime(df['DATA'])
    df.set_index('DATA', inplace=True)

    # Preenche valores ausentes com a média histórica
    df = preencher_media_historica(df, usar_mediana)

    # Exporta o resultado respeitando o formato decimal com vírgula
    df.to_csv('output.csv', index=True, sep=';', decimal=',', encoding='latin1')
    print("Arquivo CSV exportado com sucesso!")

    return df


In [9]:
df_rascunho = load_df([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

Arquivo CSV exportado com sucesso!


In [11]:
df = df_rascunho.copy()

df.temperatura_max = (df.temperatura_max + df.temperatura_min) / 2

df.rename(columns={df.columns[6]: 'temperatura_med'}, inplace=True)

# Exclui colunas indesejadas
df = df.drop(df.columns[[7]], axis=1)

In [12]:
# Calcular a janela de precipitação (precisa ser feita primeiro)
df = calcular_janela_precipitacao(df, janela=5)  # Usa uma janela de 5 dias

In [13]:
df = df[(df['HORA'] == '16:00') | (df['HORA'] == '1600 UTC')]

# Padroniza os horários
df.loc[df['HORA'] != '16:00', 'HORA'] = '16:00'
df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M').dt.time

<ipython-input-13-2ca129208add>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M').dt.time


In [14]:
df = calcula_nesterov(df)
# Aplicar a classificação ao DataFrame
df['faixa_risco'] = df['nesterov'].apply(classificar_risco)
df["Estacao"] = df.index.map(estacao_do_ano)

In [15]:
df.to_csv('planilha_final.csv', index=True, sep=';', decimal=',', encoding='latin1')